<a href="https://colab.research.google.com/github/fberanizo/spelling-correction/blob/master/tentativa_de_pretreino_do_BERT_pt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Grammatical Error Correction: FineTune in BERT PT

**Nome: Fabio Beranizo Lopes**<br>
**Nome: Luiz Pita Almeida**

Usaremos o modelo BERT pré-treinado no dataset  BrWaC (Brazilian Web as Corpus) <br>

Faremos um finetuning com os corpus:
- textos literários brasileiros
- floresta
- CorpusTCC do Nilc
- CHAVE Folha
- CHAVE Público

Avaliaremos o resultado no corpus do Nilc.

## Mount Drive and Unzip Corpus

In [1]:
from google.colab import drive
drive.mount("/content/drive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
!time unzip -n -qq "/content/drive/My Drive/pt_corpus/CorpusTCC.zip"
!time unzip -n -qq "/content/drive/My Drive/pt_corpus/brazilianliteture.zip"
!time unzip -n -qq "/content/drive/My Drive/pt_corpus/corpusNilc.zip"
!time unzip -n -qq "/content/drive/My Drive/pt_corpus/CHAVEFolha.zip"
!time unzip -n -qq "/content/drive/My Drive/pt_corpus/CHAVEPublico.zip"


real	0m4.805s
user	0m0.710s
sys	0m0.547s

real	0m3.871s
user	0m0.531s
sys	0m0.071s

real	0m11.445s
user	0m0.006s
sys	0m0.026s

real	0m16.330s
user	0m10.939s
sys	0m1.618s

real	1m11.130s
user	0m15.813s
sys	0m2.970s


## Load Corpus

### Load Floresta

In [3]:
import nltk
nltk.download('floresta')
from nltk.corpus import floresta

floresta_data = [' '.join(seq) for seq in floresta.sents()]

floresta_vocab = list(set(floresta.words()))

print('Tamanho do vocab:', len(floresta_vocab))
print('Número de sentenças:', len(floresta_data))
for sentence in floresta_data[:10]:
    print(sentence)

[nltk_data] Downloading package floresta to /root/nltk_data...
[nltk_data]   Unzipping corpora/floresta.zip.
Tamanho do vocab: 29421
Número de sentenças: 9266
Um revivalismo refrescante
O 7_e_Meio é um ex-libris de a noite algarvia .
É uma de as mais antigas discotecas de o Algarve , situada em Albufeira , que continua a manter os traços decorativos e as clientelas de sempre .
É um_pouco a versão de uma espécie de « outro lado de a noite , a meio caminho entre os devaneios de uma fauna periférica , seja de Lisboa , Londres , Dublin ou Faro e Portimão , e a postura circunspecta de os fiéis de a casa , que de ela esperam a música « geracionista de os 60 ou de os 70 .
Não deixa de ser , em os tempos que correm , um certo « very_typical algarvio , cabeça de cartaz para os que querem fugir a algumas movimentações nocturnas já a caminho de a ritualização de massas , de o género « vamos todos a o Calypso e encontramos- nos em a Locomia .
E assim , a os 2,5 milhões que o Ministério_do_Planeame

### Load Brazilian Portuguese Corpus

In [4]:
import os
path = "/content/Brazilian_Portugese_Corpus"
br_liteture_corpus = []
for root, dirs, files in os.walk(path):
    for file in files:
        if file.endswith(".txt"):
            path_file = os.path.join(root, file)
            with open(path_file, encoding="latin1") as txt_file:
                br_liteture_corpus.append(
                    txt_file.read().splitlines())

br_liteture = [seq for filecorpus in br_liteture_corpus
               for seq in filecorpus if seq is not '']
words = [word for seq in br_liteture for word in seq.split()]
br_liteture_vocab = list(set(words))
print('Tamanho do vocab:', len(br_liteture_vocab))
print('Número de sentenças:', len(br_liteture))
for sentence in br_liteture[:10]:
    print(sentence)

Tamanho do vocab: 233317
Número de sentenças: 689490
www.nead.unama.br
1
Universidade da Amazônia
A Moreninha
de Joaquim Manuel Macedo
NEAD  NÚCLEO DE EDUCAÇÃO A DISTÂNCIA
Av. Alcindo Cacela, 287  Umarizal
CEP: 66060-902
Belém  Pará
Fones: (91) 210-3196 / 210-3181


### Load NILC TCC Corpus

In [1]:
import os
import glob
path = "/content/CorpusTCC"
corpusTCC = []
for txt_file in glob.glob(os.path.join(path, "*.txt")):
    with open(txt_file, encoding="latin1") as txt:
        corpusTCC.append(txt.read().splitlines())
corpusTCC_data = [seq for filecorpus in corpusTCC
                  for seq in filecorpus if seq is not '']
words = [word for seq in corpusTCC_data for word in seq.split()]
corpusTCC_vocab = list(set(words))
print('Tamanho do vocab:', len(corpusTCC_vocab))
print('Número de sentenças:', len(corpusTCC_data))
for sentence in corpusTCC_data[:10]:
    print(sentence)

Tamanho do vocab: 8411
Número de sentenças: 732
É indiscutível que os computadores têm uma importância crescente para a nossa sociedade cada vez mais complexa, e desde a invenção da escrita e da imprensa, nada tem causado tanto impacto social e estimulado tantas mudanças no mundo quanto o uso de computadores.
A visão de que o uso de computadores na educação promove o aumento da produtividade, ensinando mais a mais gente em menos tempo, é o que baseava inicialmente as investigações sobre o uso de informática na educação. Atualmente, o que se enfatiza é o provável potencial oferecido pelo uso do computador no desenvolvimento de habilidades cognitivas que proporcionam melhores meios de ensino e aprendizagem. 
Os primeiros sistemas computacionais usados para o ensino são comumente conhecidos como CAI (Computer Assisted Instruction) e têm como objetivo principal a transmissão de informação (instrução) sobre determinado assunto (domínio) para o usuário/estudante. As informações são transmiti

### Load CHAVEFolha

In [6]:
import glob
from bs4 import BeautifulSoup

corpus_folha = []

path = "/content/folha94"
for sgml_file in glob.glob(os.path.join(path, "*.sgml")):
    with open(sgml_file, encoding='latin1') as sgml:
        text = sgml.read()
        soup = BeautifulSoup(text)
        text = [str(seq)[7:] for seq in soup.find_all('text')
                if str(seq) is not '</text>' or str(seq) is not '<text>']
        corpus_folha.append(text)

path = "/content/folha95"
for sgml_file in glob.glob(os.path.join(path, "*.sgml")):
    with open(sgml_file, encoding='latin1') as sgml:
        text = sgml.read()
        soup = BeautifulSoup(text)
        text = [str(seq)[7:] for seq in soup.find_all('text')
                if str(seq) is not '</text>' or str(seq) is not '<text>']
        corpus_folha.append(text)

folha_data = [seq[:-7] for filecorpus in corpus_folha
              for seq in filecorpus if seq is not '']
folha_data = [subseq for seq in folha_data for subseq in seq.split('\n')
              if subseq is not '']
words = [word for seq in folha_data for word in seq.split()]
folha_vocab = list(set(words))
print('Tamanho do vocab:', len(folha_vocab))
print('Número de sentenças:', len(folha_data))
for sentence in folha_data[:10]:
    print(sentence)

Tamanho do vocab: 1012666
Número de sentenças: 1186651
Dois jogos definem hoje os finalistas do Campeonato Brasileiro de futebol deste ano. Em Campinas, o Guarani recebe o Palmeiras às 17h precisando vencer por dois gols.
O outro jogo é Corinthians e Atlético-MG. Sua realização no Morumbi só foi definida na noite de anteontem. O clube de São Paulo e a Federação Paulista de Futebol reivindicavam adiar a partida para amanhã, alegando falta de segurança. A CBF confirmou o jogo às 17h. O estádio terá 70% da arquibancada interditada. Esporte
Embora ainda sem confirmações ou anúncios oficiais, vão surgindo nomes que deverão compor o gabinete de Fernando Henrique Cardoso. O elenco que se desenha, com base no que parece ser uma combinação de critérios técnicos e políticos, é no geral respeitável.
A escolha do titular da Fazenda, Pedro Malan, trouxe tranquilidade à economia. Um dos arquitetos do Real, de estilo sóbrio e diplomático, Malan acena com a preservação do plano, sem sobressaltos ou pr

### Load CHAVEPúblico

In [7]:
corpus_publico = []

path = "/content/publico94"
for sgml_file in glob.glob(os.path.join(path, "*.sgml")):
    with open(sgml_file, encoding='latin1') as sgml:
        text = sgml.read()
        soup = BeautifulSoup(text)
        text = [str(seq)[7:] for seq in soup.find_all('text')
                if str(seq) is not '</text>' or str(seq) is not '<text>']
        corpus_publico.append(text)

path = "/content/publico95"
for sgml_file in glob.glob(os.path.join(path, "*.sgml")):
    with open(sgml_file, encoding='latin1') as sgml:
        text = sgml.read()
        soup = BeautifulSoup(text)
        text = [str(seq)[7:] for seq in soup.find_all('text')
                if str(seq) is not '</text>' or str(seq) is not '<text>']
        corpus_publico.append(text)

publico_data = [seq[:-7] for filecorpus in corpus_publico
                for seq in filecorpus if seq is not '']
publico_data = [subseq for seq in publico_data for subseq in seq.split('\n')
               if subseq is not '']
words = [word for seq in publico_data for word in seq.split()]
publico_vocab = list(set(words))
print('Tamanho do vocab:', len(publico_vocab))
print('Número de sentenças:', len(publico_data))
for sentence in publico_data[:10]:
    print(sentence)

Tamanho do vocab: 1258391
Número de sentenças: 1090825
Do 6º para o 7º ano em dois quilómetros
O João andou até ao ano passado na Escola Preparatória de Odivelas, a frequentar o sexto ano de escolaridade, que terminou com aproveitamento. Com a escola a dois passos de casa e um sucesso educativo óbvio, horário da parte da manhã, amigos feitos e hábitos adquiridos, foi com surpresa que ouviu a funcionária da escola informar a mãe de que não tinha vaga na escola de Odivelas -- que até passou para C++S este ano -- porque as vagas existentes estavam preenchidas pelos alunos «com prioridade», nas quais ele, João, não estava contemplado.
Portanto, e porque o João não era nem um aluno com deficiências, nem um aluno excepcionalmente bom, era transferido para a Secundária da Póvoa de Santo Adrião, a mais de dois quilómetros de distância. O João, com os seus 13 anos, passou assim a frequentar uma escola «de madeira», sem os «luxos» a que ele estava habituado, isto é, sem cantina ou ginásio, e, pa

### Load NILC Corpus

In [8]:
import glob
path = "/content/corpusNilc"
nilc_corpus = []
for txt_file in glob.glob(os.path.join(path, "*.txt")):
    with open(txt_file) as txt:
        nilc_corpus.append(txt.read().splitlines())
nilc_data = [seq for filecorpus in nilc_corpus
             for seq in filecorpus if seq is not '']
words = [word for seq in nilc_data for word in seq.split()]
nilc_vocab = list(set(words))
print('Tamanho do vocab:', len(nilc_vocab))
print('Número de sentenças:', len(nilc_data))
for sentence in nilc_data[:10]:
    print(sentence)

Tamanho do vocab: 4742
Número de sentenças: 1582
﻿
Animais selvagens são animais que vivem nas selvas ou florestas.
O macaco, a onça, o leão, o tigre, etc. são animais selvagens.
Muitos animais selvagens são ferozes.
Nas matas brasileiras vivem muitos animais selvagens.
Os animais que são criados pelo homem são chamados animais domésticos:
Os animais domésticos precisam de cuidados especiais para ter saúde e não transmitir doenças.
﻿
Tiradentes, cujo nome era Joaquim José da Silva Xavier, queria separar o Brasil de Portugal.
Como castigo, seus companheiros foram presos e Tiradentes foi enforcado.


## Join Corpus

In [9]:
train_corpus = floresta_data + br_liteture + corpusTCC_data + \
               folha_data + publico_data
vocab = list(set(floresta_vocab + br_liteture_vocab + corpusTCC_vocab + \
                 folha_vocab + publico_vocab))

test_corpus = nilc_corpus

print('Número de sentenças em train_corpus:', len(train_corpus))
print('tamanho do vocab de train_corpus:', len(vocab))
print('Número de sentenças em test_corpus:', len(test_corpus))

Número de sentenças em train_corpus: 2976964
tamanho do vocab de train_corpus: 2017559
Número de sentenças em test_corpus: 181


## Olhando o Vocab

In [10]:
# ime vocab https://www.ime.usp.br/~pf/dicios/index.html
# 200296 palavras
ime_words_path = "/content/drive/My Drive/PF-Correcao/br-utf8.txt"
with open(ime_words_path) as file: # Use file to refer to the file object
   ime_vocab = file.read().splitlines()

In [11]:
prob_wrong = [word for word in vocab[:1000] if word not in ime_vocab]
print(len(prob_wrong))

950


In [12]:
prob_wrong[:50]

['grega).',
 "Poppins',",
 'RDP»,',
 'penaliza-me',
 '11/9)',
 'kings"',
 'Twenty-fifth',
 'Marcou-me',
 'l\'Editoria",',
 '«Ficas-te',
 'Furnas.',
 'Ion,',
 'impossible',
 'alívio."',
 '«variando',
 'motocultivadora!',
 'obrigados».',
 'centralista.',
 'volve,',
 '«Moçambiquero-te».',
 '"debaclê"',
 'Corrido,',
 'Jankélévitch',
 'sonora",',
 '«Jumpers»,',
 'Cay.',
 '65-57,',
 'Chaves-Académica',
 'carimbador',
 'substituídas,',
 '("Aristogatas",',
 '«sendo,',
 'UNREO',
 'Keneally).',
 'gradual»,',
 'consturado',
 '«Paz,',
 'Dryzun',
 'sentindo"',
 '(MRND)',
 'editais."',
 '«arrogar-se',
 'Yangtze.',
 'factos"],',
 'Uberaba',
 'artística_,',
 'Futebol).',
 '(eliminatória)',
 "Citizen's",
 'Giro,']

## Olhando o Tokenizer

In [6]:
try:
    from transformers import BertTokenizer, BertForMaskedLM
except:
    !pip3 install -q transformers
    from transformers import BertTokenizer, BertForMaskedLM

In [14]:
debug_subset = folha_data[:10]
tokenizer = BertTokenizer.from_pretrained(
    "neuralmind/bert-base-portuguese-cased")

for sentence in debug_subset:
    print(sentence)
    print(tokenizer.tokenize(sentence))


Dois jogos definem hoje os finalistas do Campeonato Brasileiro de futebol deste ano. Em Campinas, o Guarani recebe o Palmeiras às 17h precisando vencer por dois gols.
['Dois', 'jogos', 'define', '##m', 'hoje', 'os', 'finalistas', 'do', 'Campeonato', 'Brasileiro', 'de', 'futebol', 'deste', 'ano', '.', 'Em', 'Campinas', ',', 'o', 'Guarani', 'recebe', 'o', 'Palmeiras', 'às', '17', '##h', 'precisa', '##ndo', 'vencer', 'por', 'dois', 'gols', '.']
O outro jogo é Corinthians e Atlético-MG. Sua realização no Morumbi só foi definida na noite de anteontem. O clube de São Paulo e a Federação Paulista de Futebol reivindicavam adiar a partida para amanhã, alegando falta de segurança. A CBF confirmou o jogo às 17h. O estádio terá 70% da arquibancada interditada. Esporte
['O', 'outro', 'jogo', 'é', 'Corinthians', 'e', 'Atlético', '-', 'MG', '.', 'Sua', 'realização', 'no', 'Mor', '##umbi', 'só', 'foi', 'definida', 'na', 'noite', 'de', 'ante', '##ont', '##em', '.', 'O', 'clube', 'de', 'São', 'Paulo', 

## Criando Dataset de entrada

- dataset 2: 10% de máscaras


In [2]:
import torch
import random

seed = 0
random.seed(seed)
torch.random.manual_seed(seed)
torch.cuda.manual_seed(seed)

With debug set = corpusTCC_data

In [3]:
from torch.utils.data import Dataset, DataLoader

db_data = corpusTCC_data
random.shuffle(db_data)
train_size = int(0.8 * len(db_data))
db_train = db_data[:train_size]
db_valid = db_data[train_size:]
print(len(db_data), len(db_train), len(db_valid))

732 585 147


### Dataset com máscaras

In [4]:
import math
class MaskedDataset(Dataset):
    def __init__(self, texts, seq_len, mask_perc=0.1):
        super().__init__()

        self.texts = texts
        self.seq_len = seq_len
        self.mask_perc = mask_perc
        self.tokenizer = BertTokenizer.from_pretrained(
            "neuralmind/bert-base-portuguese-cased")
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        text = self.texts[idx]
        masked_text, mask_idxs = self.masking(text)
        encoded = self.tokenize(masked_text)
        labels_id = self.tokenize(text)
        return masked_text, encoded['input_ids'].squeeze(), \
               encoded['attention_mask'].squeeze(), mask_idxs, \
               text, labels_id['input_ids'].squeeze(), \
               labels_id['attention_mask'].squeeze() 

    def tokenize(self, text):
        return self.tokenizer.encode_plus(text=text,
                                          max_length=self.seq_len,
                                          pad_to_max_length=True,
                                          return_tensors='pt')
    def masking(self, text):
        words = text.split()
        n_words = len(words)
        if n_words >= self.seq_len:
            n_words = self.seq_len
        n_masks = int(math.ceil(self.mask_perc*n_words))
        masks_idxs = sorted(random.sample(range(0, n_words), n_masks))
        for idx in masks_idxs:
            words[idx] = self.tokenizer.mask_token
        return ' '.join(words), masks_idxs



In [18]:
test = MaskedDataset(['Eu gosto de pizza'], 9)
input_text, input_id, attention_mask, masked_words_pos, label, label_id, \
    label_att_mask = test[0]

print('Entrada')
print(input_text)
print('Label')
print(label)
print('Masked word:')
print([label.split()[pos] for pos in masked_words_pos])

print('Entrada ids')
print(input_id)
print('Label ids')
print(label_id)

print('Entrada mascara de atenção')
print(attention_mask)
print('Label mascara de atenção')
print(label_att_mask)

Entrada
[MASK] gosto de pizza
Label
Eu gosto de pizza
Masked word:
['Eu']
Entrada ids
tensor([  101,   103, 10303,   125, 13779, 15616,   102,     0,     0])
Label ids
tensor([  101,  3396, 10303,   125, 13779, 15616,   102,     0,     0])
Entrada mascara de atenção
tensor([1, 1, 1, 1, 1, 1, 1, 0, 0])
Label mascara de atenção
tensor([1, 1, 1, 1, 1, 1, 1, 0, 0])


In [19]:
test = MaskedDataset(db_train, 200)
input_text, input_id, attention_mask, masked_words_pos, label, label_id, \
    label_att_mask = test[0]

print('Entrada')
print(input_text)
print('Label')
print(label)
print('Masked word:')
print([label.split()[pos] for pos in masked_words_pos])

print('Entrada ids')
print(input_id)
print('Label ids')
print(label_id)

print('Entrada mascara de atenção')
print(attention_mask)
print('Label mascara de atenção')
print(label_att_mask)

Entrada
[MASK] aplicações em sistemas hipermídia abertos como [MASK] de software, segundo os princípios da engenharia de software, demandam modelos que sintetizem e suportem a [MASK] de [MASK] desenvolvimento. Com a popularidade [MASK] WEB e o aumento da demanda pela utilização de aplicações hipermídia que se integrem de [MASK] natural a diferentes ambientes, nota-se a necessidade de [MASK] voltados à especificação dessas aplicações fornecendo suporte, principalmente, a interoperabilidade.
Label
As aplicações em sistemas hipermídia abertos como produtos de software, segundo os princípios da engenharia de software, demandam modelos que sintetizem e suportem a automatização de seu desenvolvimento. Com a popularidade da WEB e o aumento da demanda pela utilização de aplicações hipermídia que se integrem de forma natural a diferentes ambientes, nota-se a necessidade de modelos voltados à especificação dessas aplicações fornecendo suporte, principalmente, a interoperabilidade.
Masked word:
[

In [7]:
train_dataset = MaskedDataset(db_train, 100)
valid_dataset = MaskedDataset(db_valid, 100)

train_loader = DataLoader(train_dataset, batch_size=32, 
                          shuffle=True, num_workers=4)
valid_loader = DataLoader(valid_dataset, batch_size=32, 
                          shuffle=False, num_workers=4)
data = next(iter(train_loader))
data

[('A flexibilidade e a facilidade [MASK] uso [MASK] [MASK] na Web têm garantido um futuro cada vez mais promissor para a utilização de sistemas de hipertexto. Porém, [MASK] a construção de hiperdocumentos envolve a montagem [MASK] milhares de [MASK] e [MASK] de milhares de links, ela se torna uma atividade que pode gerar muitas informações inconsistentes. Essa [MASK] acrescida dos problemas inerentes à utilização de hipertextos, conhecidos por "desorientação" do usuário e "sobrecarga cognitiva" durante a navegação pelo hipertexto, faz com [MASK] a manutenção seja uma atividade complexa. Além [MASK] com um volume enorme de informações a serem analisadas, a manutenção se torna uma atividade muitas vezes negligenciada.',
  '- A aceitação da computação paralela está intimamente ligada à possibilidade [MASK] portabilidade direta de programas entre [MASK] heterogêneos; neste contexto, [MASK] alguns autores, [MASK] MPI pode tornar-se um padrão de grande importância no futuro da computação, [M

Modelo

In [8]:
try:
    import pytorch_lightning as pl
except:
    !pip3 install --quiet pytorch_lightning
    import pytorch_lightning as pl

from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning import Trainer
from torch.optim import Adam
import torch.nn.functional as F

In [29]:
import html
class MaskedBertFineTune(pl.LightningModule):
    def __init__(self, train_loader, valid_loader):
        super().__init__()
        self.bert = BertForMaskedLM.from_pretrained(
            "neuralmind/bert-base-portuguese-cased"
        )
        self.tokenizer = BertTokenizer.from_pretrained(
            "neuralmind/bert-base-portuguese-cased"
        )
        self.ext_trainloader = train_loader
        self.ext_validloader = valid_loader
    
    def forward(self, x):
        inputs, input_ids, input_mask, masked_idx, \
        targets, target_ids, target_mask = x
        outputs = self.bert(input_ids=input_ids.to('cuda'), 
                            attention_mask=input_mask.to('cuda'),
                            masked_lm_labels=target_ids.to('cuda'))
        if self.training:
            loss, predicted_scores = outputs[:2]
            return loss, predicted_scores, inputs, targets
        else:
            return outputs, inputs, targets

        
    
    def training_step(self, batch, targets):
        loss, predicted_scores, inputs, targets = self(batch)

        return {"loss": loss, "log": {"loss": loss}}
    
    def validation_step(self, batch, batch_idx):
        outputs, inputs, targets = self(batch)
        predicts = [
            html.unescape(
                self.tokenizer.decode(output_ids,
                                      skip_special_tokens=False,
                                      clean_up_tokenization_spaces=False)
            ) for output_ids in outputs]
        return {"loss": loss, "predicts": predicts, "inputs": inputs, \
                "targets": targets}
    
    def training_epoch_end(self, outputs):
        avg_loss = torch.stack([x["loss"] for x in outputs]).mean()

        return {"log": {"train_loss": avg_loss}} 
    
    def validation_epoch_end(self, outputs):
        avg_loss = torch.stack([x["loss"] for x in outputs]).mean()

        origs = sum([list(x["inputs"]) for x in outputs], [])
        trues = sum([list(x["targets"]) for x in outputs], [])
        preds = sum([list(x["predicts"]) for x in outputs], [])

        n = random.choice(range(len(trues)))
        print(f"\Input: {origs[n]}\nTarget: {trues[n]}\nPrediction: {preds[n]}\n")

        return {"log": {"train_loss": avg_loss}}
    
    def train_dataloader(self):
        return self.ext_trainloader

    def val_dataloader(self):
        return self.ext_validloader

    def configure_optimizers(self):
        return Adam(self.parameters())


In [30]:
db_model = MaskedBertFineTune(train_loader, valid_loader)
experiment_name = 'testingTCC'
model_folder = os.path.join('logs', experiment_name)
os.makedirs(model_folder, exist_ok=True)
ckpt_path = os.path.join(model_folder, "-{epoch}")

# Callback initialization
checkpoint_callback = ModelCheckpoint(prefix=experiment_name, 
                                      filepath=ckpt_path, 
                                      mode="max")
logger = TensorBoardLogger('logs', experiment_name)
# PL Trainer initialization
trainer = Trainer(gpus=1,
                  checkpoint_callback=checkpoint_callback, 
                  early_stop_callback=False,
                  logger=logger,
                  max_epochs=1, 
                  fast_dev_run=False)
trainer.fit(db_model)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]

  | Name | Type            | Params
-----------------------------------------
0 | bert | BertForMaskedLM | 109 M 


TypeError: ignored

In [ ]:
out[0]

In [12]:
out[1]

tensor([[[ -9.4054,  -8.6048,  -8.2732,  ...,  -9.5237,  -8.1939,  -7.9912],
         [ -7.0730,  -8.1739,  -7.8436,  ...,  -6.0889, -10.5214,  -6.3005],
         [ -9.5637, -10.7983, -10.3706,  ...,  -6.7952, -11.4531,  -7.2657],
         ...,
         [-12.0864, -12.4690, -10.3375,  ..., -11.2548, -10.7601, -10.7757],
         [ -8.8735,  -9.0423,  -7.7689,  ...,  -6.3143,  -7.1291,  -7.5126],
         [ -9.4006,  -8.6055,  -8.2716,  ...,  -9.5150,  -8.1946,  -7.9779]],

        [[ -8.1549,  -7.6893,  -8.2547,  ...,  -9.0093,  -6.3883,  -8.4629],
         [ -8.6314,  -7.4760,  -5.9166,  ...,  -6.6270,  -5.1959,  -6.9518],
         [ -6.5959, -10.0313,  -7.6038,  ...,  -5.0818,  -8.5121, -10.4720],
         ...,
         [ -6.1700,  -6.1061,  -6.3812,  ...,  -6.8128,  -6.3264,  -7.2589],
         [ -7.3913,  -7.6567,  -6.2813,  ...,  -7.9391,  -5.7155,  -9.1154],
         [ -8.9512,  -9.4239,  -8.2928,  ...,  -7.4815,  -6.5856,  -7.7257]],

        [[ -9.0201,  -8.3548,  -9.0330,  ...

In [19]:
del db_model

In [14]:
out[3]

('A flexibilidade e a facilidade de uso de hiperdocumentos na Web têm garantido um futuro cada vez mais promissor para a utilização de sistemas de hipertexto. Porém, quando a construção de hiperdocumentos envolve a montagem de milhares de páginas e centenas de milhares de links, ela se torna uma atividade que pode gerar muitas informações inconsistentes. Essa inconsistência, acrescida dos problemas inerentes à utilização de hipertextos, conhecidos por "desorientação" do usuário e "sobrecarga cognitiva" durante a navegação pelo hipertexto, faz com que a manutenção seja uma atividade complexa. Além disso, com um volume enorme de informações a serem analisadas, a manutenção se torna uma atividade muitas vezes negligenciada. ',
 '- A aceitação da computação paralela está intimamente ligada à possibilidade de portabilidade direta de programas entre sistemas heterogêneos; neste contexto, segundo alguns autores, o MPI pode tornar-se um padrão de grande importância no futuro da computação, tan

In [18]:
out[4][2]

IndexError: ignored